In [2]:
!pip install playwright openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 19.2 MB/s eta 0:00:00


In [5]:
!playwright install


172.5 MiB [] 0% 0.0s172.5 MiB [] 0% 47.2s172.5 MiB [] 0% 22.8s172.5 MiB [] 0% 15.3s172.5 MiB [] 0% 8.7s172.5 MiB [] 1% 6.4s172.5 MiB [] 1% 5.6s172.5 MiB [] 2% 5.2s172.5 MiB [] 2% 4.7s172.5 MiB [] 3% 4.4s172.5 MiB [] 3% 4.1s172.5 MiB [] 4% 3.9s172.5 MiB [] 4% 3.8s172.5 MiB [] 5% 3.8s172.5 MiB [] 5% 3.9s172.5 MiB [] 5% 4.0s172.5 MiB [] 6% 3.9s172.5 MiB [] 6% 3.8s172.5 MiB [] 7% 3.5s172.5 MiB [] 8% 3.2s172.5 MiB [] 9% 3.1s172.5 MiB [] 10% 3.0s172.5 MiB [] 11% 2.7s172.5 MiB [] 12% 2.5s172.5 MiB [] 13% 2.5s172.5 MiB [] 14% 2.4s172.5 MiB [] 15% 2.3s172.5 MiB [] 16% 2.2s172.5 MiB [] 17% 2.1s172.5 MiB [] 19% 2.0s172.5 MiB [] 20% 1.9s172.5 MiB [] 21% 1.8s172.5 MiB [] 22% 1.7s172.5 MiB [] 24% 1.7s172.5 MiB [] 25% 1.6s172.5 MiB [] 26% 1.6s172.5 MiB [] 27% 1.6s172.5 MiB [] 28% 1.5s172.5 MiB [] 30% 1.5s172.5 MiB [] 31% 1.4s172.5 MiB [] 33% 1.3s172.5 MiB [] 34% 1.3s172.5 MiB [] 36% 1.2s172.5 MiB [] 37% 1.2s172.5 MiB [] 38% 1.1s172.5 MiB [] 40% 1.1s172.5 MiB [] 41% 1.1s172.5 MiB [] 42% 1.0s172.5 MiB 

In [ ]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
import openai
import json

nest_asyncio.apply()

openai.api_key = "APIKEY"

materials_reference = {
    "Electronics": [
        "Computers, Laptops, Tablets",
        "Monitors, TVs (CRT & Flat Screen)",
        "Cell Phones, Smartphones",
        "Printers, Copiers, Fax Machines",
        "Audio/Video Equipment",
        "Gaming Consoles",
        "Small Appliances (Microwaves, Toasters, etc.)",
        "Computer Peripherals (Keyboards, Mice, Cables, etc.)"
    ]
}

def classify_materials(raw_description):
    prompt = f"""
You are a smart extractor classifying recycling materials.

Here is a raw text description of accepted materials at a recycling center:

"{raw_description}"

Based on the below categories and sub-materials, return:
1. A list of matched "materials_category"
2. A list of matched "materials_accepted"

Your response must be in this JSON format:
{{
  "materials_category": [...],
  "materials_accepted": [...]
}}

Categories and materials:
{json.dumps(materials_reference, indent=2)}

Only include valid exact matches from the above lists.
"""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return json.loads(response.choices[0].message.content)

async def scrape_earth911():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True, slow_mo=50)  # headless=False to fix timing/iframe issues
        page = await browser.new_page()
        await page.goto("https://search.earth911.com/", timeout=60000)
        await page.wait_for_timeout(8000)

        # Use keyboard navigation as a backup if selectors fail
        await page.keyboard.press("Tab")
        await page.keyboard.type("Electronics")
        await page.keyboard.press("Tab")
        await page.keyboard.type("10001")
        await page.keyboard.press("Enter")

        await page.wait_for_timeout(10000)

        facilities = await page.locator(".location-info").all()
        output = []

        for facility in facilities[:3]:
            business_name = await facility.locator("h2").inner_text()
            address = await facility.locator(".address").inner_text()
            try:
                update = await facility.locator(".last-updated").inner_text()
            except:
                update = "Unknown"
            try:
                accepted = await facility.locator(".accepted-materials").inner_text()
            except:
                accepted = ""

            classification = classify_materials(accepted)

            output.append({
                "business_name": business_name.strip(),
                "last_update_date": update.strip(),
                "street_address": address.strip().replace("\n", ", "),
                "materials_category": classification["materials_category"],
                "materials_accepted": classification["materials_accepted"]
            })

        await browser.close()
        with open("output.json", "w") as f:
            json.dump(output, f, indent=2)
        print("Done. Data saved to output.json.")

await scrape_earth911()

Done. Data saved to output.json.


In [11]:
import json

with open("output.json") as f:
    data = json.load(f)

for item in data:
    print(json.dumps(item, indent=2))
